In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math
from config import *

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'Price', 'FPL_pos', 'FPL_points',
       'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == NEXT_GAMEWEEK]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
24,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,8.099314
34,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.278045
95,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.676491
122,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.405799
133,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.225135
193,Bruno Borges Fernandes,5.0,2023-24,Manchester Utd,Brighton,1.0,1860.949707,1839.788574,8.4,MID,NaN,5.589880
195,Bukayo Saka,5.0,2023-24,Arsenal,Everton,0.0,1928.752808,1682.138794,8.7,MID,NaN,5.580100
258,Jarrod Bowen,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,7.1,MID,NaN,5.388615
265,Guglielmo Vicario,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,5.1,GK,NaN,5.358993
269,Moussa Diaby,5.0,2023-24,Aston Villa,Crystal Palace,1.0,1828.641479,1768.190308,6.6,MID,NaN,5.351518


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((500, 12), (59, 12), (172, 12), (213, 12), (56, 12))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
24,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,8.099314
34,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.278045
95,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.676491
122,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.405799
133,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.225135


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df_original = df.copy()

In [13]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [14]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [15]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [16]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [17]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Shola Shoretire, FWD, 4.4, 0.62, Manchester Utd) - (fixtures: 1.0)'

In [18]:
# for p in playersDB:
#     print(p.__str__())

In [19]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [20]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [21]:
budget = 100

In [22]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
        p.benched = True
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [23]:
best_teams = []
best_team = None
best_team_score = 0
best_bench = None

for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team_for_formation = best_full_teams(smallDB, [formation], budget - bench_price)
    
    if best_team_for_formation[0][1] > best_team_score:
        best_team = best_team_for_formation
        best_team_score = best_team_for_formation[0][1]
        best_bench = bench
        
for p in best_team[0][2]:
    p.benched = True
for p in best_bench:
    best_team[0][2].append(p)
        
    

[3, 4, 3] 1 2 1 0
(Matt Turner, GK, 4.0, 1.1, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 1.3, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.47, Manchester City) - (fixtures: 1.0)
(Isaac Hayden, MID, 4.4, 1.81, Newcastle Utd) - (fixtures: 1.0)
16.200000000000003
With formation [3, 4, 3]: 64.21  | (price = 83.10000000000001)
(Guglielmo Vicario, GK, 5.1, 5.36, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 6.23, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 4.86, Liverpool) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.6, Chelsea) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 7.28, Tottenham) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.68, Liverpool) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 6.41, West Ham) - (fixtures: 1.0)
(Moussa Diaby, MID, 6.6, 5.35, Aston Villa) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 8.1, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 4.73, Luton) - (fixtures: 1.0)
(Alexander Isak, FWD

In [24]:
for p in best_team[0][2]:
    print(p.__str__())

(Guglielmo Vicario, GK, 5.1, 5.36, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 6.23, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 4.86, Liverpool) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.6, Chelsea) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 7.28, Tottenham) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.68, Liverpool) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 6.41, West Ham) - (fixtures: 1.0)
(Jarrod Bowen, MID, 7.1, 5.39, West Ham) - (fixtures: 1.0)
(Moussa Diaby, MID, 6.6, 5.35, Aston Villa) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 8.1, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 4.73, Luton) - (fixtures: 1.0)
(Matt Turner, GK, 4.0, 1.1, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 1.3, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.47, Manchester City) - (fixtures: 1.0)
(Shola Shoretire, FWD, 4.4, 0.62, Manchester Utd) - (fixtures: 1.0)


In [25]:
%%time
# best_team = best_full_teams(smallDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
f"Best team score: {best_team[0][1]}"

'Best team score: 64.99'

In [27]:
best_team[0]

([3, 5, 2],
 64.99,
  <knapsack_problem.player.Player at 0x227f2a38b20>])

In [28]:
def best_team_to_df(best_team_squad, df):
    names = [p.name for p in best_team_squad]
    print(names)
    custom_fpl_order = ["FWD", "MID", "DEF", "GK"]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    output = df[df["Name_original"].isin(names)].reset_index()
    output["First_eleven"] = True
    output.loc[11:15, "First_eleven"] = False
    output = output.sort_values(by=["First_eleven", "FPL_pos", "Price"], ascending=False)
    
    return output

In [29]:
best_squad = best_team_to_df(best_team[0][2], df_original)
best_squad

['Guglielmo Vicario', 'Destiny Udogie', 'Trent Alexander-Arnold', 'Axel Disasi', 'James Maddison', 'Mohamed Salah', 'James Ward-Prowse', 'Jarrod Bowen', 'Moussa Diaby', 'Erling Haaland', 'Carlton Morris', 'Matt Turner', 'Tariq Lamptey', 'Sergio Gómez', 'Shola Shoretire']


,index,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction,First_eleven
6,265,Guglielmo Vicario,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,5.1,GK,NaN,5.358993,True
8,485,Trent Alexander-Arnold,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,7.9,DEF,NaN,4.860766,True
10,637,Axel Disasi,5.0,2023-24,Chelsea,Bournemouth,0.0,1774.226440,1659.008667,5.1,DEF,NaN,4.599286,True
4,133,Destiny Udogie,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,4.7,DEF,NaN,6.225135,True
2,95,Mohamed Salah,5.0,2023-24,Liverpool,Wolves,0.0,1961.337402,1708.177246,12.5,MID,NaN,6.676491,True
1,34,James Maddison,5.0,2023-24,Tottenham,Sheffield United,1.0,1846.944458,1630.776367,7.8,MID,NaN,7.278045,True
5,258,Jarrod Bowen,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,7.1,MID,NaN,5.388615,True
7,269,Moussa Diaby,5.0,2023-24,Aston Villa,Crystal Palace,1.0,1828.641479,1768.190308,6.6,MID,NaN,5.351518,True
3,122,James Ward-Prowse,5.0,2023-24,West Ham,Manchester City,1.0,1807.522339,2087.353760,6.1,MID,NaN,6.405799,True
0,24,Erling Haaland,5.0,2023-24,Manchester City,West Ham,0.0,2087.353760,1807.522339,14.1,FWD,NaN,8.099314,True


In [30]:
best_squad.to_csv("predictions/best_squad.csv")